In [1]:
from torchvision import datasets, transforms
import torch.utils.data
import torch
import sys
import argparse
import matplotlib.pyplot as plt
# from utils import * 
from utils import *
import open3d as o3d
from models import *
from collections import OrderedDict
import os, shutil, gc
from tqdm import tqdm_notebook
import matplotlib.cm as cm
import numpy as np

In [ ]:
%matplotlib notebook

In [ ]:
def get_quadrant(point):
    if point[0] >= 0. and point[1] >= 0. :
        return 0
    elif point[0] <= 0. and point[1] >= 0. : 
        return 1
    elif point[0] <= 0. and point[1] <= 0. : 
        return 2
    elif point[0] >= 0. and point[1] <= 0. : 
        return 3
    else :
        raise Exception('invalid input %s', point) 


def passed_origin(x_t, x_t1):
    if get_quadrant(x_t1) == 3 and get_quadrant(x_t) == 0: 
        return True
    else : 
        return False


def fit_quadrant(points, quadrant, desired_amt):
    
    
    points = np.asarray(points)
    slots = []
    slot_size = np.pi / (2 * desired_amt)
    for i in range(int(desired_amt)) : slots.append([])
    if quadrant == 0: 
        points = points[::-1]
    elif quadrant == 1 : 
        points[:, 0] = - points[:, 0]
    elif quadrant == 2 :
        points = points[::-1] 
        points[:, 0] = - points[:, 0]
        points[:, 1] = - points[:, 1]
    elif quadrant == 3 : 
        points[:, 1] = - points[:, 1]

    # import pdb; pdb.set_trace()
    for point in points :
        angle = np.arctan(point[1] / (point[0]+0.000001))
        index = min(int(angle / slot_size), desired_amt - 1)
        slots[int(index)].append(point)

    for i in range(len(slots)):
        if len(slots[i]) == 0 : 
            slots[i] = np.array([0., 0., 0., 0.])
        else :
            full_slot = np.asarray(slots[i])
            slots[i] = full_slot.mean(axis=0)

    points = np.asarray(slots)
    if quadrant == 0: 
        points = points[::-1]
    elif quadrant == 1 : 
        points[:, 0] = - points[:, 0]
    elif quadrant == 2 : 
        points = points[::-1]
        points[:, 0] = - points[:, 0]
        points[:, 1] = - points[:, 1]
    elif quadrant == 3 : 
        points[:, 1] = - points[:, 1]

    return points

def parse_velo(velo):
    # points closer to the origin (0,0,0) are at the end of the point cloud.
    # invert the point cloud such that we begin near the origin. 
    
    # returns: a H x 4 x ? array, split into quadrants
    velo = velo[::-1]
    lines = []
    current_point = velo[0]
    print(current_point)
    current_quadrant = get_quadrant(current_point)
    print(current_quadrant)
    current_line = [[], [], [], []]
    quadrant_switches = 0
    pass_org = 0
    for point in velo :
        point_quadrant = get_quadrant(point)
        
        if passed_origin(current_point, point):
            lines.append(current_line)
            current_line = [[], [], [], []]
#             print("passed_origin")
            pass_org += 1
        current_line[point_quadrant].append(point)
        current_quadrant = point_quadrant
        current_point = point
    print("Passed origin : {} times".format(pass_org))
    return lines


def setmatch(lines,lenLines):
    arr=[[np.array([0,0,0,0]),np.array([0,0,0,0])]]
    if len(lines) > lenLines:
        return lines[:lenLines]
    else:
        for i in range(abs(len(lines)-lenLines)):
            lines.append(arr)
    return lines

def process_velo(velo, points_per_layer, stop=False):
    
    lenLines=RANGE_IMAGE_HEIGHT
    lines = parse_velo(velo)
#     inverse = quad_to_pc_inv(lines)
#     lines = lines[2:-1]
#     print(lines[])
#     print((lines[0]))
#     raise SystemError
    print(len(lines), flush=True)
    if(len(lines)!=lenLines):
        lines=setmatch(lines,lenLines)
    print(len(lines), flush=True)
    if len(lines) != RANGE_IMAGE_HEIGHT : raise Exception('invalid nb un of lines')
    out_tensor = np.zeros((RANGE_IMAGE_HEIGHT, points_per_layer, 4))
    if stop:
        import pdb; pdb.set_trace()
        x = 1
    for j in range(len(lines)):
        line = lines[j]
        out_line = np.zeros((points_per_layer, 4))
        for i in range(len(line)):
            if(len(line[i])==0):
                line[i]=[np.array([0.0,0.0,0.0,0.0])]
            gridded = fit_quadrant(line[i], i, points_per_layer / 4)
            out_tensor[j][i*int(points_per_layer/4):(i+1)*int(points_per_layer/4), :] = gridded[::-1]

    return out_tensor#, inverse


def quad_to_pc_inv(lines, th=3.):
    # lines is a 63 x 4 array, where each slot has an array of 4d/3d points
    # goal : get an array of points that fills empty spaces
    points = []
    for i in range(len(lines)) :
        line = lines[i] 
        distance = []
        for quad in line : 
            for point in quad : 
                x, y, z = point[:3]
                distance.append(x**2 + y**2)
        distance = np.array(distance)
        std = distance.std()
        sorted_indices = np.argsort(distance)
        median_index = sorted_indices[int(sorted_indices.shape[0]*0.95)]
        median = distance[median_index]

        for quad in line : 
            for point in quad : 
                x, y, z = point[:3]
                dist = x ** 2 + y ** 2 
                if dist < median and (median/dist-1.) > th:#*std : 
                    # blocked point --> scale to get real pt
                    scale = np.sqrt(median / dist)
                    scaled = scale * point
                    points.append(scaled)


    return np.array(points)


In [ ]:
processed_carla_lidar = process_velo(carla_lidar, RANGE_IMAGE_WIDTH)

In [ ]:
processed_ati_lidar = process_velo(ati_lidar, RANGE_IMAGE_WIDTH)

In [ ]:
processed_ati_lidar_sort = process_velo(ati_lidar_sort, RANGE_IMAGE_WIDTH)

In [ ]:
processed_ati_lidar_sort[:,:,:3]

In [ ]:
RANGE_IMAGE_HEIGHT = 16
RANGE_IMAGE_WIDTH  = 1024

In [ ]:
old_npy_file_path = "/home/saby/Projects/ati/data/data/datasets/Carla/16beam-Data/small_map/pair_transform_single/static_begin_npy/100.npy"
carla_lidar = np.load(old_npy_file_path, allow_pickle=True)
# processed_lidar = process_velo(raw_lidar, RANGE_IMAGE_WIDTH)

In [ ]:
carla_lidar.shape

In [ ]:
# npy_file_path = "/home/saby/Projects/ati/data/data/datasets/Real_World/dynamic/2/_out_begin_npy/340.npy"
# ati_lidar = np.load(npy_file_path, allow_pickle=True)

pcd_path = "/home/saby/Projects/ati/data/data/datasets/Real_World/dynamic/1/_out/340.pcd"
this_pcd = o3d.io.read_point_cloud(pcd_path)
ati_lidar = np.asarray(this_pcd.points)
ati_lidar = np.concatenate((ati_lidar, np.zeros((ati_lidar.shape[0],1))), axis=1)
ati_lidar.shape
# processed_lidar = process_velo(raw_lidar, RANGE_IMAGE_WIDTH)

In [ ]:
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import copy

In [ ]:
nn_phi = NearestNeighbors(n_neighbors=1)
# laser_angles = [-15, 1, -13, 3, -11, 5, -9, 7, -7, 9, -5, 11, -3, 13, -1, 15] # in degrees
laser_angles = [-15, -13, -11, -9, -7, -5, -3, -1, 1, 3, 5, 7, 9, 11, 13, 15] # in degrees
laser_angles_ = np.array(laser_angles)
laser_angles_arr = np.array([np.array([angle,0]) for angle in laser_angles])
nn_phi.fit(laser_angles_arr)

In [ ]:
def appendSpherical_np(xyz):
#     ptsnew = np.hstack((xyz, np.zeros(xyz.shape)))
    xy = xyz[:,0]**2 + xyz[:,1]**2
    r_arr = np.sqrt(xy + xyz[:,2]**2)
    phi_arr = np.rad2deg(np.arctan2(np.sqrt(xy), xyz[:,2])) -90 # for elevation angle defined from Z-axis down
    theta_arr = np.rad2deg(np.arctan2(xyz[:,1], xyz[:,0]))
    return r_arr, theta_arr, phi_arr

def correct_point_ordering(ati_lidar, TRANSFORM_LIKE_CARLA=True):
    # Put everything in dataframe
    df_ati_lidar = pd.DataFrame(ati_lidar).rename(columns={0:'x', 1:'y', 2:'z', 3:'i'})
    r_arr, theta_arr, phi_arr = appendSpherical_np(ati_lidar[:,:3])
    df_ati_lidar['r'] = r_arr
    df_ati_lidar['theta'] = theta_arr
    df_ati_lidar['phi'] = phi_arr
    
    # Get correct phi (vertical angles)
    phi_arr = phi_arr.reshape(-1,1)    
    d_list, idx_list = nn_phi.kneighbors(np.concatenate((phi_arr, np.zeros(phi_arr.shape)), axis=1))
    df_ati_lidar['new_phi'] = laser_angles_[idx_list][:,0]
    # df_ati_lidar['neg_new_phi'] = -laser_angles_[idx_list][:,0]
    
    # Sort as per carla order
    df_ati_lidar_sort = df_ati_lidar.sort_values(by=['new_phi', 'theta'], ascending=True)
    ati_lidar_sort = df_ati_lidar_sort.values[:,:3]
    
    if TRANSFORM_LIKE_CARLA:
        x_carla = copy.deepcopy(ati_lidar_sort[:,1])
        y_carla =  -copy.deepcopy(ati_lidar_sort[:,0])
        z_carla =  -copy.deepcopy(ati_lidar_sort[:,2])
        ati_lidar_sort[:,0] = x_carla
        ati_lidar_sort[:,1] = y_carla
        ati_lidar_sort[:,2] = z_carla
    
    ati_lidar_sort = np.concatenate((ati_lidar_sort, ati_lidar[:,3].reshape((-1,1))), axis=1)
    return ati_lidar_sort

In [ ]:
ati_lidar_sort =  correct_point_ordering(ati_lidar)
ati_lidar.shape, ati_lidar_sort.shape

In [ ]:
processed_lidar = process_velo(carla_lidar, RANGE_IMAGE_WIDTH)

processed_lidar = process_velo(ati_lidar, RANGE_IMAGE_WIDTH)

processed_lidar = process_velo(ati_lidar_sort, RANGE_IMAGE_WIDTH)

In [ ]:
plt.figure()
plt.grid()
start_idx = -10240
end_idx = -1

plt.scatter(ati_lidar_sort[start_idx:end_idx,0], ati_lidar_sort[start_idx:end_idx,1], c=cm.rainbow((np.arange(end_idx-start_idx)/(end_idx-start_idx))))

In [ ]:
plt.figure()
plt.scatter(np.arange(ati_lidar_sort.shape[0]), appendSpherical_np(ati_lidar_sort[:,:3])[1].round())

In [ ]:
plt.figure()
plt.scatter(np.arange(ati_lidar_sort.shape[0]), appendSpherical_np(ati_lidar_sort[:,:3])[2].round())

In [ ]:
plt.figure()
plt.grid()
start_idx = -10240

end_idx = -1
plt.scatter(carla_lidar[start_idx:end_idx,0], carla_lidar[start_idx:end_idx,1], c=cm.rainbow((np.arange(end_idx-start_idx)/(end_idx-start_idx))))

# points are ordered first by beam then by rotation angle
# initial lines for far off sky points
# last lines form near origin ground points

In [ ]:
plt.figure()
plt.scatter(np.arange(carla_lidar.shape[0]), appendSpherical_np(carla_lidar[:,:3])[1].round())

In [ ]:
plt.figure()
plt.scatter(np.arange(carla_lidar.shape[0]), appendSpherical_np(carla_lidar[:,:3])[2].round())

In [ ]:
assert False

In [ ]:
ati_lidar.round?

In [ ]:
dataset_val = np.load("/home/saby/Projects/ati/data/data/datasets/Real_World/dynamic/1/_out_npz/0.npz")#, allow_pickle=True)

In [ ]:
dataset_val['arr_0'].sum()

In [ ]:
dataset_val.shape

In [ ]:
frame_sum = dataset_val.sum(-1).sum(-1).sum(-1)
frame_sum.shape

In [ ]:
zero_frames = frame_sum == 0.0
(zero_frames).sum()

In [ ]:
list(dataset_val[~zero_frames])[0]

In [ ]:
dataset_val_real = np.load("/home/saby/Projects/ati/data/data/datasets/Real_World/pair_transform/dynamic_out_npy/1.npy", allow_pickle=True)
# dataset_val_carla = np.load("/home/saby/Projects/ati/data/data/datasets/Carla/16beam-Data/small_map/testing/8/_out_out_npy/0.npy", allow_pickle=True)

In [ ]:
dataset_val = preprocess(dataset_val_real, 120)

In [ ]:
dataset_val.shape

In [ ]:
dataset_val[0]

In [ ]:
frame_img = dataset_val[100]

In [ ]:
frame_img.shape

In [ ]:
frame_img_3d = from_polar_np(np.expand_dims(frame_img, axis=0))[0]
frame_img_3d.shape

In [ ]:
some_pcd = o3d.geometry.PointCloud()
some_arr = frame_img_3d.reshape((3,-1)).T
some_pcd.points = o3d.utility.Vector3dVector(some_arr)

In [ ]:
def draw_pcd(pcd, where='opn_nb'):
    if where is 'opn_nb':
        visualizer = o3d.JVisualizer()
        visualizer.add_geometry(pcd)
        visualizer.show()
    elif where is 'opn_view':
        o3d.visualization.draw_geometries([pcd], width=1280, height=800)
    elif where is 'mat_3d':
        plt.figure()
        pts = np.asarray(pcd.points)
        plt.scatter(pts[:,0], pts[:,1], pts[:,2])
        plt.grid()
        plt.show()
    elif where is 'mat_2d':
        plt.figure()
        pts = np.asarray(pcd.points)
        plt.scatter(pts[:,0], pts[:,1])
        plt.grid()
        plt.show()

In [ ]:
draw_pcd(some_pcd)